In [ ]:
!unzip mlops.zip

# <span style='color :#40be46' > Let's prepare the environment </span>


✨ <span style='color : #fae253' > TASK </span> ✨

Please perform the following steps:
1. Navigate to *Administration --> Repositories*.
2. Create a remote HuggingFaceML repository in your Artifactory project. This repository will be used to cache the models HuggingFace.
3. Create a remote Python repository in your Artifactory project. This repository will be used to cache the python packages that are used in the code.
4. Run the following code to test the connectivity to Artifactory:

In [ ]:
# TODO test connectivity to Artifactory

# <span style='color :#40be46' > Lab1: Caching HuggingFace models in Artifactory </span>

## Configure HuggingFace client to work through Artifactory

✨ <span style='color : #fae253' > TASK </span> ✨

Please open your Artifactory instance and navigate to your newly created *remote HuggingFaceML* repository and click on "Set Me Up" in the top bar on the right.
1. Copy the *token* and paste it in the cell below, replacing the <IDENTITY_TOKEN> placeholder
2. Copy the *HF_ENDPOINT __value__* and paste it in the cell below, replacing the <PATH> placeholder

 👀 The next cell sets the environment variables such that the huggingface client which we'll use later will not fetch the model from the hugging_face hub, but rather from Artifactory.

In [ ]:
# Replace the <IDENTITY-TOKEN> placeholder with the token you generated in the JFrog Platform SetMeUp.
%env HF_TOKEN=<IDENTITY_TOKEN>
# Replace the <PATH> placeholder with the path to your ML Model Management repository in Artifactory, found in the JFrog Platform SetMeUp.
%env HF_ENDPOINT=<PATH>

%env HF_HUB_ETAG_TIMEOUT=86400

## Download the required Python packages through Artifactory

✨ <span style='color : #fae253' > TASK </span> ✨

Please open your Artifactory instance and navigate to your newly created *remote PyPi repository* :
1. Navigate to *Application --> Artifactory --> Artifacts* and search for your repository.
2. Click on "Set Me Up" and the top bar on the right.
1. Navigate to the "Install" tab.
2. Copy the index-url value and paste it in the cell below, replacing the <ARTIFACTORY_PIP_REPOSITORY_URL> placeholder

👀 The next cell downloads and installs the required python pacages while caching it in Artifactory.

In [ ]:
# Replace <ARTIFACTORY_PIP_REPOSITORY_URL> with the URL pointing to your pip repository found in the the JFrog Platform Set-Me-Up.
!pip3 install huggingface_hub ultralytics -i <ARTIFACTORY_PIP_REPOSITORY_URL>

## Python imports

In [ ]:
from huggingface_hub import snapshot_download, HfApi
from huggingface_hub.utils import HfHubHTTPError

import json

from ultralytics import YOLO

import cv2
import random

from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

from google.colab.patches import cv2_imshow

## Download the pre-trained model

In [ ]:
# Load the model and processor
model_name = "shirabendor/YOLOV8-oiv7"
weights = "yolov8m-oiv7.pt"
config_file = "./config.json"

try:
    snapshot_download(repo_id=model_name, allow_patterns=weights, local_dir=".")
except HfHubHTTPError as e:
    print("\n\n\U0001F6A8\U0001F6A8\U0001F6A8\U0001F6A8 Xray blocked model download due to violation of the 'Block-Unknown' license policy.\U0001F6A8\U0001F6A8\U0001F6A8\U0001F6A8\n\n")



✨ <span style='color : #fae253' > TASK </span> ✨

Let's check Artifactory to review the outcome.

Please open your Artifactory instance and navigate to your newly created *remote HuggingfaceML repository*
1. Navigate to *Application --> Artifactory --> Artifacts"
2. Find your newly created HuggingFaceML remote repository.
3. Expand the repository and verify the YOLOV8 model is cached inside the repository.

## Helper Functions

👀 The following cell defines some helper functions

In [ ]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

def overlay_image_alpha(img, img_overlay, pos):
    """Overlay img_overlay on top of img at the position specified by pos."""
    x, y = pos

    # Image ranges
    y1, y2 = max(0, y), min(img.shape[0], y + img_overlay.shape[0])
    x1, x2 = max(0, x), min(img.shape[1], x + img_overlay.shape[1])

    # Overlay ranges
    y1o, y2o = max(0, -y), min(img_overlay.shape[0], img.shape[0] - y)
    x1o, x2o = max(0, -x), min(img_overlay.shape[1], img.shape[1] - x)

    # Exit if nothing to do
    if y1 >= y2 or x1 >= x2 or y1o >= y2o or x1o >= x2o:
        return img

    # Blend overlay within the determined ranges
    img_crop = img[y1:y2, x1:x2]
    img_overlay_crop = img_overlay[y1o:y2o, x1o:x2o]

    # Split the alpha channel and the color channels
    if img_overlay_crop.shape[2] == 4:  # Ensure the overlay has an alpha channel
        img_overlay_color = img_overlay_crop[:, :, :3]
        alpha_mask = img_overlay_crop[:, :, 3] / 255.0

        alpha_inv = 1.0 - alpha_mask

        for c in range(0, 3):
            img_crop[:, :, c] = (alpha_mask * img_overlay_color[:, :, c] +
                                 alpha_inv * img_crop[:, :, c])
    else:
        img_crop[:, :, :] = img_overlay_crop

    return img


# Inference function

👀 The following cell defines the inference function. The model is configured to identify people.

In [ ]:
import logging

logging.getLogger("ultralytics").setLevel(logging.ERROR)  # see docs for other logging levelsv

model = YOLO(weights)

# Load the frog image with alpha channel
frog_img = cv2.imread('./frog_plane.png', cv2.IMREAD_UNCHANGED)
frog_height, frog_width = frog_img.shape[:2]
label_color_map = {}


def infere():

    with open(config_file, 'r') as f:
        config = json.load(f)

    classes        = config['classes']
    target_classes = config['target_classes']

    filename = take_photo()
    frame = cv2.imread(filename)

    frame_height, frame_width = frame.shape[:2]
    frog_x = frame_width  # Start position for the frog (outside the screen on the right)
    # Set the frog image at the top of the screen
    frog_y = 0

    results = model.predict(source=frame,
                            show=False,
                            classes=classes,
                            conf=0.05,
                            max_det=2)

    # Extracting the names of detected classes
    boxes = results[0].boxes

     # Draw bounding boxes
    for box in boxes:
        label = model.names[int(box.cls)]
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Convert to integer coordinates
        if label not in label_color_map:
            label_color_map[label] = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        # Draw bounding box around detected person
        cv2.rectangle(frame, (x1, y1), (x2, y2), label_color_map[label], 3)  # Colored box

        if int(box.cls) in target_classes:
          # Overlay the frog image
          cv2.putText(frame, "Frog", (x1, y1 - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, label_color_map[label], 2)
          frame = overlay_image_alpha(frame, frog_img, (frog_x, frog_y))

        else:
          cv2.putText(frame, label.title(), (x1, y1 - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, label_color_map[label], 2)

    cv2_imshow(frame)



In [ ]:
try:
    infere()
except Exception as e:
    cv2.destroyAllWindows()
    raise e


# <span style='color :#40be46' > Lab2: Securing models </span>

## Block malicious model with Xray

✨ <span style='color : #fae253' > TASK </span> ✨

Let's configure XRay to scan our HuggingFaceML remote repository.

#### Complete the following steps:

***Add the HuggingFaceML remote repository to XRay index:***

1. Navigate to *Administration --> Xray Settings --> Indexed Resources and click on `+ Add a Reposotiry`
2. Search for your repository name on the right hand side search box, and drag it to the left hand side table. Click `Save`

***Create a watch***

1. Navigate to *Application --> Xray --> Watches & Policies", click on the "Watches" tab and click on `+ New Watch`.
2. In the form, give your watch a name of your choice.
3. Click on `Add Repositories`. In the form that opens, select your repository on the right hand side and drag it to the left hand side. Click `Save`.

***Create a policy***

1. Navigate to *Application --> Xray --> Watches & Policies" and click on `+ New Policy`.
2. In the form, give your policy a name of your choice and leave the "Security" policy ytype selected. Click on `Next`.
3. In the next form, on the right hand side fom, give your rule a name of your choice and in the Rule Type dropdown select "Malicious Packages" . On the left hand side select the "Block download" action. Click `Save Rule`.
4. Back on the policy wizard, click `Next` and select your created watch from the previous step.
5. Click `Save Policy`.



In [ ]:
try:
    snapshot_download(repo_id="MustEr/best_model_for_identifying_frogs")
except HfHubHTTPError as e:
    print("\n\n\U0001F6A8\U0001F6A8\U0001F6A8\U0001F6A8 Xray blocked model download due to violation of the 'Malicious Package' policy.\U0001F6A8\U0001F6A8\U0001F6A8\U0001F6A8\n\n")

# <span style='color :#40be46' > Lab3: Uploading updated model to a local repository and deploying with Qwak </span>

## Train the model to identify airplanes

👀 Until now, our model detected only people. Now, we will 'train' it to identify other objects, specifically airplanes.

Due to time constraints, our training function does not actually train on additional images. Instead, we'll just change the model configuration. Check the "config.json" file before and after the training to see the difference.


In [ ]:
def train(object_to_detect):

    if not object_to_detect in model.names.values():
        print(f"'{object_to_detect}' is not a valid YOLOv8 object. Hint: try Frog")
        return

    reverse_dict = {name: idx for idx, name in model.names.items()}
    class_id = reverse_dict.get(object_to_detect, None)

    with open(config_file, 'r') as file:
        config = json.load(file)

    target_classes = config['target_classes']

    # Add the new class number to the classes list if it's not already present
    if class_id not in config['classes']:
        config['classes'].append(class_id)
        config['classes'].extend([cls for cls in target_classes if cls not in config['classes']])


    # Save the updated config back to the file
    with open(config_file, 'w') as file:
        json.dump(config, file, indent=4)


In [ ]:
train("Frog")

## Run inference again

In [ ]:
try:
    infere()
except Exception as e:
    cv2.destroyAllWindows()
    raise e


## Upload to HF local

👀 Now that we have a new, trained model, we need to upload it to Artifactory HugginigFaceML local repository.

✨ <span style='color : #fae253' > TASK </span> ✨

Please perform the following steps:
1. Navigate to *Administration --> Repositories*.
2. Create a **local** HuggingFaceML repository in your Artifactory project. This repository will be used to cache the models HuggingFace.
3. Navigate to "Application --> Artifactory --> Artifacts" and find your newly created local repository. Click on "Set Me Up" and the top bar on the right.
4. Copy the *token* and paste it in the cell below, replacing the <IDENTITY_TOKEN> placeholder
5. Copy the HF_ENDPOINT value and paste it in the cell below, replacing the <PATH> placeholder


In [ ]:
# Replace the <IDENTITY-TOKEN> placeholder with the token you generated in the JFrog Platform SetMeUp.
%env HF_TOKEN=<IDENTITY-TOKEN>

# Replace the <PATH> placeholder with the path to your ML Model Management repository in Artifactory, found in the JFrog Platform SetMeUp.
%env HF_ENDPOINT=<PATH>

%env HF_HUB_DOWNLOAD_TIMEOUT=86400
%env HF_HUB_ETAG_TIMEOUT=86400

In [ ]:
from huggingface_hub import HfApi
import os

# Initialize API with the custom endpoint
api = HfApi(endpoint=os.getenv("HF_ENDPOINT"))

# Upload folder to the specified repository
api.upload_folder(
    folder_path=".",
    repo_id="<MODEL_NAME>",   # Replace with a name for your model
    repo_type="model"
)

### Check the results in Artifactory

✨ <span style='color : #fae253' > TASK </span> ✨

Let's check Artifactory to review the outcome.


1. Please open your Artifactory instance Navigate to *Artifactory --> Artifacts* tab.
2. Find your newly created *local HuggingFaceML repository*.
3. Expand the repository and verify the YOLOV8 model is cached inside the repository, including the updated configuration file.

## Deploy with Qwak

We'll start by installing the qwak SDK.

✨ <span style='color : #fae253' > TASK </span> ✨

1. Replace <ARTIFACTORY_PIP_REPOSITORY_URL> with the URL pointing to your pip repository found in the the JFrog Platform Set-Me-Up (you can take it from the cell in the first lab)
2. Create a personal API key in the Qwak platform:
    - Go to [Quak Platform](https://app.qwak.ai/)
    - On the left hand side menu, Navigate to *Settings --> Personal API Keys*
    - Click on `Generate API Key`
    - Copy the API key generated and replace the below <QWAK_PERSONAL_API_KEY> placeholder with it

In [ ]:
# Replace <ARTIFACTORY_PIP_REPOSITORY_URL> with the URL pointing to your pip repository found in the the JFrog Platform Set-Me-Up.
!pip3 install qwak-sdk -i <ARTIFACTORY_PIP_REPOSITORY_URL>

# Replace <QWAK_PERSONAL_API_KEY> with your Qwak personal key from the qwak platform.
!qwak configure --api-key <QWAK_PERSONAL_API_KEY>

# Test successful connection
!qwak projects list

### Build the Qwak model

✨ <span style='color : #fae253' > TASK </span> ✨

In order to build and deploy the model through the Qwak platform, we'll need to first find the model-id.
1. In the [Quak Platform](https://app.qwak.ai/) Navigate to *Models*.
2. Select your project and click on your model.
3. Copy your model-id from the information bar under the title with the model name (you'll have a clickable copy icon once you hover on it)
4. Replace the <MODEL_ID> placeholder below with your model_id.


In [ ]:
!qwak models build --model-id <MODEL_ID> .

### Check your model build status (can take up to 10 minutes)

### Deploy your model

✨ <span style='color : #fae253' > TASK </span> ✨

1. In the [Quak Platform](https://app.qwak.ai/) Navigate to *Models*.
2. Select your project and click on your model.
3. Under the *Builds* tab, identify your build and click `Deploy`
4. Select `Realtime`
5. On the next screen, no need to change anything, click on `Deploy Model`


In [ ]:
models.shira-jfrog1.qwak.ai/v1/yolo_test_1/default/predict()